# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686602


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:20,  2.78s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:07,  2.42s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:44,  1.73s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:29,  1.23s/it]

...

Rendering models:  71%|███████   | 22/31 [00:08<00:01,  5.00it/s]

Rendering models:  81%|████████  | 25/31 [00:08<00:00,  6.43it/s]

Rendering models:  87%|████████▋ | 27/31 [00:08<00:00,  7.89it/s]

Rendering models:  97%|█████████▋| 30/31 [00:08<00:00,  9.29it/s]

froboy                                0.016773
not-logged-in-20a275b53abec8db9f41    1.771165
CThomas                               0.005194
cschwefl                              0.000994
Dynamatt                              0.000728
mkehrli                               0.000547
not-logged-in-c3e2bb63f9ec3bdeea91    0.000542
mtajnai                               0.000735
justsomenoodles                       0.001206
browny811                             0.001151
not-logged-in-dddbb97cfd84354416f5    0.002110
Planetme                              0.003234
tominas                               0.007775
Lavadude                              0.004886
not-logged-in-15498ebaa693dd5a198d    0.000797
not-logged-in-88d5e424af552d0f2955    0.011807
Vickyb123                             0.001594
atoderov                              0.000801
rmacmi2                               0.000745
nicholas.                             0.011450
FR4N1O                                0.005757
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())